In [49]:

import os
import json
import time
import base64
import requests

import pandas as pd
pd.set_option('display.max_rows', None)

import warnings
warnings.filterwarnings('ignore')

import numpy as np
from datetime import datetime
import time

def tr(data):
    data['previous_close'] = data['close'].shift(1)
    data['high-low'] = abs(data['high'] - data['low'])
    data['high-pc'] = abs(data['high'] - data['previous_close'])
    data['low-pc'] = abs(data['low'] - data['previous_close'])

    tr = data[['high-low', 'high-pc', 'low-pc']].max(axis=1)

    return tr

def atr(data, period):
    data['tr'] = tr(data)
    atr = data['tr'].rolling(period).mean()

    return atr

def supertrend(df, period=10, atr_multiplier=3):
    hl2 = (df['high'] + df['low']) / 2
    df['atr'] = atr(df, period)
    df['upperband'] = hl2 + (atr_multiplier * df['atr'])
    df['lowerband'] = hl2 - (atr_multiplier * df['atr'])
    df['in_uptrend'] = True

    for current in range(1, len(df.index)):
        previous = current - 1
        
        if df['close'][current] > df['upperband'][previous]:
            df['in_uptrend'][current] = True
        elif df['close'][current] < df['lowerband'][previous]:
            df['in_uptrend'][current] = False
        else:
            df['in_uptrend'][current] = df['in_uptrend'][previous]

            if df['in_uptrend'][current] and df['lowerband'][current] < df['lowerband'][previous]:
                df['lowerband'][current] = df['lowerband'][previous]

            if not df['in_uptrend'][current] and df['upperband'][current] > df['upperband'][previous]:
                df['upperband'][current] = df['upperband'][previous]
        
    return df


def get_data(symbol, bar):
    
    d = requests.get(f'https://www.okx.com/api/v5/market/index-candles?instId={symbol}&bar={bar}').json()
    data = pd.DataFrame(d['data'],columns=['timestamp','open','high','low','close','confirm'])
    data['timestamp'] = pd.to_datetime(data['timestamp'],utc=True, unit='ms')
    for i in ['open','high','low','close']:
        data[i] = data[i].astype(float)
    data = data.set_index('timestamp').sort_index()
    return data

def get_current_trend(symbol, bar):
    data = get_data(symbol, bar)
    data = supertrend(data)
    return bool(data.iloc[len(data)-1]['in_uptrend'])

In [48]:
data = get_data('SOL-USDT', '15m')
data=supertrend(data)
data.iloc[len(data)-1]

open               105.49
high                105.5
low                105.14
close              105.37
confirm                 0
previous_close     105.49
high-low             0.36
high-pc              0.01
low-pc               0.35
tr                   0.36
atr                 0.635
upperband         106.995
lowerband         103.415
in_uptrend          False
Name: 2024-02-21 07:29:42.400000+00:00, dtype: object

In [50]:

from aevo import AevoClient
from clientConfig import CLIENT_CONFIG

from datetime import datetime
from loguru import logger as log



# Add Credentials
aevo = AevoClient(**CLIENT_CONFIG)


In [53]:
aevo.rest_get_data("positions")

{'account': '0x00000038383f81B2C6153C37053Bd6c3201f42b4', 'positions': []}

In [57]:
aevo.get_markets('ETH')

[{'instrument_id': '1',
  'instrument_name': 'ETH-PERP',
  'instrument_type': 'PERPETUAL',
  'underlying_asset': 'ETH',
  'quote_asset': 'USDC',
  'price_step': '0.01',
  'amount_step': '0.01',
  'min_order_value': '10',
  'max_order_value': '5000000',
  'max_notional_value': '2000000',
  'mark_price': '2903.057891',
  'index_price': '2902.856105',
  'is_active': True,
  'max_leverage': '20'},
 {'instrument_id': '12592',
  'instrument_name': 'ETH-29NOV23-1875-P',
  'instrument_type': 'OPTION',
  'underlying_asset': 'ETH',
  'quote_asset': 'USDC',
  'price_step': '0.1',
  'amount_step': '0.1',
  'min_order_value': '0.01',
  'max_order_value': '2000000',
  'max_notional_value': '10000000',
  'mark_price': '0',
  'forward_price': '0',
  'index_price': '2902.856105',
  'is_active': False,
  'option_type': 'put',
  'expiry': '1701244800000000000',
  'strike': '1875'},
 {'instrument_id': '12596',
  'instrument_name': 'ETH-29NOV23-1875-C',
  'instrument_type': 'OPTION',
  'underlying_asset': 

In [58]:
get_current_trend('ETH-USDT', '15m')

False

In [54]:


c = 0
def run_bot(symbol,symbol_id,is_trading_up,amount,ticks):
    
    
    global c
    c+=1
    aevo.rest_cancel_all_orders()

    print(f"[CLEAR] clear existing positions {datetime.now().isoformat()}")
    while len(aevo.rest_get_data("positions")['positions'])>0:
        pos = aevo.rest_get_data("positions")['positions'][0]
        side = pos['side']
        pos_amount = float(pos['amount'])

        if side == 'buy':
            aevo.rest_create_market_order(symbol_id,False,pos_amount)
        else:
            aevo.rest_create_market_order(symbol_id,True,pos_amount)
        time.sleep(5)
        aevo.rest_cancel_all_orders()

    
    print(f"[OREDER] epoch {c} time {datetime.now().isoformat()}")

    direction = 'asks' if is_trading_up else 'bids'
    aevo.rest_create_market_order(symbol_id,is_trading_up,amount)
    aevo.rest_create_order(symbol_id,not is_trading_up, float(aevo.get_orderbook(symbol)[direction][ticks][0]),amount) 
    open_time = datetime.now()

    print(f"opentime {open_time.isoformat()} checking all position filled")

    while len(aevo.rest_get_data("positions")['positions'])>0 or len(aevo.rest_get_open_orders()) > 0 :
        print(f"{datetime.now().isoformat()} position not filled")
        if (datetime.now() - open_time).seconds > 60*20:
            print(f"[TIMEOUT] timeout {datetime.now().isoformat()}")
            break
        time.sleep(10)

    

while True:
    run_bot('ETH-PERP',1,get_current_trend('ETH-USDT', '15m'),0.1,10)
    time.sleep(1)

[CLEAR] clear existing positions 2024-02-21T15:54:52.976840


2024-02-21 15:54:53.327 | INFO     | aevo:sign_order:517 - {'name': 'Aevo Mainnet', 'version': '1', 'chainId': '1'}


[OREDER] epoch 1 time 2024-02-21T15:54:53.327736


2024-02-21 15:54:53.977 | INFO     | aevo:sign_order:517 - {'name': 'Aevo Mainnet', 'version': '1', 'chainId': '1'}
2024-02-21 15:54:53.989 | INFO     | aevo:rest_create_order:196 - {'maker': '0x00000038383f81B2C6153C37053Bd6c3201f42b4', 'is_buy': True, 'instrument': 5197, 'limit_price': '104996000', 'amount': '3000000', 'salt': '6312432644', 'signature': '0xbe1efb829f63818fd571b993496375d3ee924da8022638994ffdc2a992e6f1be23d3f197ebf6be146a0cb4b4d0bdf72e15e3d025b5e7ac10efb62820b15e2b911c', 'post_only': True, 'reduce_only': False, 'close_position': False, 'timestamp': 1708502093}


opentime 2024-02-21T15:54:54.314871 checking all position filled
2024-02-21T15:54:54.627897 position not filled
2024-02-21T15:55:04.970937 position not filled
2024-02-21T15:55:15.286506 position not filled
2024-02-21T15:55:25.621922 position not filled
[CLEAR] clear existing positions 2024-02-21T15:55:37.995475


...

2024-02-21 19:57:38.766 | INFO     | aevo:sign_order:517 - {'name': 'Aevo Mainnet', 'version': '1', 'chainId': '1'}


[OREDER] epoch 59 time 2024-02-21T19:57:38.766447


2024-02-21 19:57:39.416 | INFO     | aevo:sign_order:517 - {'name': 'Aevo Mainnet', 'version': '1', 'chainId': '1'}
2024-02-21 19:57:39.433 | INFO     | aevo:rest_create_order:196 - {'maker': '0x00000038383f81B2C6153C37053Bd6c3201f42b4', 'is_buy': True, 'instrument': 5197, 'limit_price': '102390000', 'amount': '3000000', 'salt': '600177010', 'signature': '0x28d68a9a02a49689215ab5b47e25bbdcaef736820c1713b9bd3c6510d59b92895587273daa042c85d764cece7440442e2bff4042a96f3a217eedf355fd1027a11b', 'post_only': True, 'reduce_only': False, 'close_position': False, 'timestamp': 1708516659}


opentime 2024-02-21T19:57:39.765379 checking all position filled
2024-02-21T19:57:40.129356 position not filled
2024-02-21T19:57:50.477086 position not filled
2024-02-21T19:58:00.806625 position not filled


KeyboardInterrupt: 

In [22]:
get_current_trend('SOL-USDT', '15m')==True

True